<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import zipfile
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pandas as pd

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [5]:
!pip uninstall -y scikit-learn xgboost
!pip install scikit-learn==1.2.2 xgboost==1.7.6

Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Successfully uninstalled scikit-learn-1.6.0
Found existing installation: xgboost 2.1.3
Uninstalling xgboost-2.1.3:
  Successfully uninstalled xgboost-2.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [2]:
# ✅ שלב 1: שכפול המאגר מה-GitHub
!git clone https://github.com/orifelszer/CrimeData.git

# ✅ שלב 2: פונקציה לפתיחת קובצי ZIP מהתיקייה המקומית
def load_zipped_csv_local(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_name = zip_ref.namelist()[0]
        with zip_ref.open(csv_name) as file:
            return pd.read_csv(file)

# ✅ שלב 3: קריאת הנתונים לאחר השכפול (נתיב מקומי)
X_train = load_zipped_csv_local('CrimeData/X_train_supervised.zip')
X_test = load_zipped_csv_local('CrimeData/X_test_supervised.zip')
y_train = load_zipped_csv_local('CrimeData/y_train_supervised.zip')
y_test = load_zipped_csv_local('CrimeData/y_test_supervised.zip')

# ✅ בדיקה שהנתונים נטענו כראוי
print(f"X_train Shape: {X_train.shape}")
print(f"X_test Shape: {X_test.shape}")
print(f"y_train Shape: {y_train.shape}")
print(f"y_train Shape: {y_train.shape}")

fatal: destination path 'CrimeData' already exists and is not an empty directory.
X_train Shape: (1257198, 10)
X_test Shape: (376971, 10)
y_train Shape: (1257198, 1)
y_train Shape: (1257198, 1)


In [3]:
# הגדרת המודל עם scale_pos_weight
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    random_state=42,
)

In [4]:
# אימון המודל
xgb_model.fit(X_train.values, y_train.values)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [5]:
# חיזוי על סט הבדיקה
y_pred = xgb_model.predict(X_test)

In [6]:
# פונקציה להערכת המודל
def evaluate_model(y_true, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "F1 Score": f1_score(y_true, y_pred, average="weighted", zero_division=0)
    }

In [8]:
# הדפסת ביצועי המודל
xgb_metrics = evaluate_model(y_test, y_pred)
print("Improved XGBoost Metrics:", xgb_metrics)

Improved XGBoost Metrics: {'Accuracy': 0.3821063158704516, 'Precision': 0.35156940510210927, 'Recall': 0.3821063158704516, 'F1 Score': 0.3136799844272996}
